# 1. AdaptivePooling

<br>
<br>
<br>


<div>
<img src='pooling.png' width='400' height='400'/>
</div>

---

> 解释

    自适应池化Adaptive Pooling与标准的Max/AvgPooling区别在于，自适应池化Adaptive Pooling会根据输入的参数来控制输出output_size，而标准的Max/AvgPooling是通过kernel_size，stride与padding来计算output_size： 

    1. stride = (input_size//output_size)
    2. kernel size = input_size - (output_size-1)*stride
    3. padding = 0


In [3]:
import torch as t
import math
import numpy as np
 
alist = t.randn(2,10,10)
 
inputsz = np.array(alist.shape[1:])
outputsz = np.array([5,5])

# 反推步长和size
stridesz = np.floor(inputsz/outputsz).astype(np.int32)
kernelsz = inputsz-(outputsz-1)*stridesz

print('stridesz:', stridesz)
print('kernelsz:', kernelsz)
 
adp = t.nn.AdaptiveAvgPool2d(list(outputsz))
avg = t.nn.AvgPool2d(kernel_size=list(kernelsz),stride=list(stridesz))
adplist = adp(alist)
avglist = avg(alist)
print('输入：')
print(alist.shape)
print('输出：')
print(adplist.shape)
print(avglist.shape)
print('验证：')
print(adplist == avglist)


stridesz: [2 2]
kernelsz: [2 2]
输入：
torch.Size([2, 10, 10])
输出：
torch.Size([2, 5, 5])
torch.Size([2, 5, 5])
验证：
tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])


In [4]:
import torch as t
import math
import numpy as np
 
alist = t.randn(2,3,9)
 
inputsz = np.array(alist.shape[2:])
outputsz = np.array([4])
 
stridesz = np.floor(inputsz/outputsz).astype(np.int32)
 
kernelsz = inputsz-(outputsz-1)*stridesz
 
adp = t.nn.AdaptiveAvgPool1d(list(outputsz))
avg = t.nn.AvgPool1d(kernel_size=list(kernelsz),stride=list(stridesz))
adplist = adp(alist)
avglist = avg(alist)
print('输入：')
print(alist.shape)
print('输出：')
print(adplist.shape)
print(avglist.shape)
print('验证：')
print(adplist==avglist)


输入：
torch.Size([2, 3, 9])
输出：
torch.Size([2, 3, 4])
torch.Size([2, 3, 4])
验证：
tensor([[[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]],

        [[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]]])


## conv2d

<br>
<br>
<br>


<div>
<img src='conv.png' width='600' height='600'/>
</div>

---

torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')


> 参数
    
    1. in_channels：输入数据的channels数量
    2. out_channels：输出数据的channels数量
    3. kernel_size：卷积核大小，tuple格式或者整数
    4. stride：卷积核移动步数
    5. padding：padding数量
    6. dilation：卷积核元素之间的间距。空洞卷积。
    7. groups：分组卷积。
    8. bias：是否加入可训练的偏执参数，True/False
    9. padding_mode:填充值
    

In [5]:
import torch

x = torch.randn(5,2,10,10)
conv = torch.nn.Conv2d(in_channels=2,out_channels=8,kernel_size=(2,2))
res = conv(x)
print(x.shape)
print(conv.weight.data.size())
print(res.shape)

torch.Size([5, 2, 10, 10])
torch.Size([8, 2, 2, 2])
torch.Size([5, 8, 9, 9])


In [46]:
# 输入数据的channels必须与卷积核设置的输入channels相同

import torch

x = torch.randn(5,3,10,10)
conv = torch.nn.Conv2d(in_channels=2,out_channels=8,kernel_size=(2,2)) 
res = conv(x)
print(x.shape)
print(conv.weight.data.size())
print(res.shape)

RuntimeError: Given groups=1, weight of size [8, 2, 2, 2], expected input[5, 3, 10, 10] to have 2 channels, but got 3 channels instead

## 2.1 groups：

  
**groups：输入in_channels被划分为groups组， 每组通道数为in_channels/groups。  
    每组需要重复计算out_channels/(n_channels/groups)次。  
    最后，在channels维concat起来得到最后的6*H_out*W_out输出**


- 当设置group=1时

In [21]:
conv = nn.Conv2d(in_channels=6, out_channels=6, kernel_size= (2,2), groups=1)
conv.weight.data.size()

torch.Size([6, 6, 2, 2])

- 当设置为group=2时

In [22]:
conv = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(2,2), groups=2)
conv.weight.data.size()

torch.Size([6, 3, 2, 2])

- 当设置group=3时

In [24]:
conv = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(2,2), groups=3)
conv.weight.data.size()

torch.Size([6, 2, 2, 2])

- 当设置group=4时

In [26]:
conv = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=1, groups=4)
conv.weight.data.size()

ValueError: in_channels must be divisible by groups

## 2.2  dilation实现空洞卷积


> 公式 ：

<div>
<img src='dilation.png' width='500' height='500'/>
</div>

---
> 图示：

<div>
<img src='dilation2.png' width='600' height='600'/>
</div>



In [7]:
import torch

x = torch.randn(5,2,12,12)
conv1 = torch.nn.Conv2d(in_channels=2,out_channels=8,kernel_size=(4,4),dilation=1)
conv2 = torch.nn.Conv2d(in_channels=2,out_channels=8,kernel_size=(4,4),dilation=2)
res1 = conv1(x)
res2 = conv2(x)
print(x.shape)

print('dilation=1:')
print(conv1.weight.data.size())
print(res1.shape)
print('dilation=2:')
print(conv2.weight.data.size())
print(res2.shape)

# filter参数量不变，但是感受野变大了

torch.Size([5, 2, 12, 12])
dilation=1:
torch.Size([8, 2, 4, 4])
torch.Size([5, 8, 9, 9])
dilation=2:
torch.Size([8, 2, 4, 4])
torch.Size([5, 8, 6, 6])
